# Modelo de tópicos como sistemas de recomendação

Na primeira parte da aula, falamos sobre sistemas de recomendação baseado em filtros colaborativos. Aqui, vamos nos focar em outro tipo de sistema de recomendação: o baseado em conteúdo. Vamos continuar falando de fatorização de matrizes para criar sistemas de recomendação, e vamos ver como a técnica `NMF` pode nos ajudar a criar um sistema de recomendação interpretável.

Bom, mas qual é a diferença desse tal de NMF em relação ao SVD? Técnicas como o SVD reduzem a esparsidade do dataset e, em geral, melhoram as predições de ratings. Entretanto, uma desvantagem clara é a falta de uma explicabilidade dos fatores gerados.

<img src="resources/imgs/NMF_PCA.png" width=700>

Fonte: [Learning the parts of objects by non-negative matrix factorization](http://www.columbia.edu/~jwp2128/Teaching/E4903/papers/nmf_nature.pdf)

Essa característica do NMF de aprender partes do objeto é muito útil à construção de um modelo de tópicos, cujo objetivo em tarefas textuais é caracterizar documentos em tópicos (conjunto de palavras).

<img src="resources/imgs/topic_model.png" width=500>

### NMF (Non-negative Matrix Factorization)

O objetivo da fatorização de matrizes usando NMF é encontrar matrizes **não negativas** ($W$ e $H$) que ao serem multiplicadas geram aproximadamente a matriz original ($X$):

$$X = W \cdot H$$

A escolha do número de tópicos (parâmetro `n_components` no [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.NMF.html)) é feita a priori, e determina uma das dimensões das matrizes $W$ e $H$.

#### **Atividade**: construa um sistema de recomendação usando o algoritmo NMF

Etapas:

1. Pré-processamento do dataset
2. Crie o modelo de tópicos:
    * construa uma matriz de input para o NMF usando o [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)
    * faça a decomposição [NMF](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.NMF.html) dessa matriz para obter a matriz que descreve os livros em tópicos e a matriz que descreve os tópicos em features (da matriz de input)
3. Use a matriz que descreve os livros em tópicos para fazer o `fit` do algoritmo [knn](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestNeighbors.html#sklearn.neighbors.NearestNeighbors)
4. Carregue os dados processados do outro notebook e encontre a matriz de usuário por tópicos
5. Dado um usuário (uma linha da matriz de usuário por tópicos) e um número $n > 0$, crie uma função que encontra as top $n$ recomendações de livros (lista de `book_id`) para ele 

** Tempo estimado para a atividade total: ** 45 minutos

**Etapa 1: ** pré-processamento

1. Carregue o dataset `resources/data/reduced/books_summary.csv` em um `pandas.DataFrame`
2. Crie uma coluna que concatena as seguintes informações: título (`title`), autores (`authors`) e tags (`tags`)
3. Normalize a coluna recém criada, colocando todo o texto em caixa baixa (_lowercase_) e usando expressão regular `REG_SYMBOLS` para remover caracteres indesejados do texto

** Tempo estimado para a atividade: ** 10 minutos

In [1]:
import pandas as pd
import re

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', -1)

In [3]:
books_df = pd.read_csv('resources/data/reduced/books_summary.csv')

In [4]:
books_df['text'] = books_df['title'] + ' ' + books_df['authors'] + ' ' + books_df['tags']

In [5]:
REG_SYMBOLS = re.compile(r'[^A-Za-z0-9 #\-]+')

In [6]:
books_df['norm_text'] = books_df['text'].str.lower().str.replace(REG_SYMBOLS, '')

* ao final dessa etapa, você deve ter uma coluna a mais no dataset, que mostra os textos concatenados e com a normalização proposta

In [7]:
books_df.head(n=1)

,book_id,goodreads_book_id,title,authors,original_publication_year,tags,text,norm_text
0,1,2767052,"The Hunger Games (The Hunger Games, #1)",Suzanne Collins,2008.0,to-read fantasy favorites currently-reading young-adult fiction books-i-own owned ya series favourites re-read adventure sci-fi-fantasy all-time-favorites default my-books reread i-own audiobook 5-stars favorite-books novels fantasy-sci-fi favorite audiobooks read-more-than-once my-library ya-fantasy teen english books ya-fiction my-favorites own-it library audio young-adult-fiction novel scifi-fantasy faves favorite-series shelfari-favorites kindle romance favourite to-buy read-in-2014 ebook contemporary 5-star coming-of-age favourite-books favs action read-in-2013 read-in-2011 finished ya-books borrowed sci-fi ya-lit science-fiction scifi sf book-club speculative-fiction ebooks survival action-adventure e-book drama thriller suspense dystopia dystopian love read-in-2012 post-apocalyptic futuristic dystopias distopian distopia teen-fiction loved read-2012 trilogy read-in-2010 dystopian-fiction 2012-reads future reviewed read-2011 ya-dystopian finished-series completed-series love-triangle suzanne-collins hunger-games the-hunger-games,"The Hunger Games (The Hunger Games, #1) Suzanne Collins to-read fantasy favorites currently-reading young-adult fiction books-i-own owned ya series favourites re-read adventure sci-fi-fantasy all-time-favorites default my-books reread i-own audiobook 5-stars favorite-books novels fantasy-sci-fi favorite audiobooks read-more-than-once my-library ya-fantasy teen english books ya-fiction my-favorites own-it library audio young-adult-fiction novel scifi-fantasy faves favorite-series shelfari-favorites kindle romance favourite to-buy read-in-2014 ebook contemporary 5-star coming-of-age favourite-books favs action read-in-2013 read-in-2011 finished ya-books borrowed sci-fi ya-lit science-fiction scifi sf book-club speculative-fiction ebooks survival action-adventure e-book drama thriller suspense dystopia dystopian love read-in-2012 post-apocalyptic futuristic dystopias distopian distopia teen-fiction loved read-2012 trilogy read-in-2010 dystopian-fiction 2012-reads future reviewed read-2011 ya-dystopian finished-series completed-series love-triangle suzanne-collins hunger-games the-hunger-games",the hunger games the hunger games #1 suzanne collins to-read fantasy favorites currently-reading young-adult fiction books-i-own owned ya series favourites re-read adventure sci-fi-fantasy all-time-favorites default my-books reread i-own audiobook 5-stars favorite-books novels fantasy-sci-fi favorite audiobooks read-more-than-once my-library ya-fantasy teen english books ya-fiction my-favorites own-it library audio young-adult-fiction novel scifi-fantasy faves favorite-series shelfari-favorites kindle romance favourite to-buy read-in-2014 ebook contemporary 5-star coming-of-age favourite-books favs action read-in-2013 read-in-2011 finished ya-books borrowed sci-fi ya-lit science-fiction scifi sf book-club speculative-fiction ebooks survival action-adventure e-book drama thriller suspense dystopia dystopian love read-in-2012 post-apocalyptic futuristic dystopias distopian distopia teen-fiction loved read-2012 trilogy read-in-2010 dystopian-fiction 2012-reads future reviewed read-2011 ya-dystopian finished-series completed-series love-triangle suzanne-collins hunger-games the-hunger-games


**Etapa 2:** criação do modelo de tópicos

* construa uma matriz de input para o NMF usando o [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)
* faça a decomposição [NMF](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.NMF.html) dessa matriz para obter a matriz que descreve os livros em tópicos e a matriz que descreve os tópicos em features (da matriz de input)

** Tempo estimado para a atividade: ** 15 minutos

In [8]:
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from sklearn.preprocessing import Normalizer

* aplique o `TfidfVectorizer` na coluna de texto criada na etapa 1 para obter uma matriz de features textuais

In [9]:
tfidf = TfidfVectorizer(max_df=0.9, min_df=0.001, sublinear_tf=True, max_features=1500, stopwords='english')
X_tfidf = tfidf.fit_transform(books_df['norm_text'].values)

* escolha a quantidade de tópicos para gerar o NMF

In [49]:
# parâmetro para NMF
n_components = 10

* calcule a decomposição `NMF` (note que ao executar o `fit_transform`, o NMF já retorna a matriz $W$ da fórmula; a matriz $H$ pode ser obtida através do atributo `.components_` do modelo NMF)

In [11]:
nmf = NMF(n_components=n_components)

In [12]:
%%time
M_bt = nmf.fit_transform(X_tfidf)

CPU times: user 532 ms, sys: 468 ms, total: 1e+03 ms
Wall time: 380 ms


In [13]:
M_tf = nmf.components_

In [14]:
M_bt.shape, M_tf.shape

((7690, 10), (10, 1500))

* normalize a matriz $W$, que relaciona os livros com os tópicos
    * para facilitar os cálculos futuros, use converta a matriz final em uma matriz esparsa `scipy.sparse.csr_matrix`. Por exemplo:
    
    ```python
    from scipy.sparse import csr_matrix
    W = csr_matrix(W)
    ```

In [33]:
norm = Normalizer()
M_bt = norm.fit_transform(csr_matrix(M_bt))

In [34]:
M_bt

<7690x10 sparse matrix of type '<class 'numpy.float64'>'
	with 39451 stored elements in Compressed Sparse Row format>

* ao final, caso deseje, você pode imprimir 10 palavras que representam cada um dos tópicos (nesse trecho de código, a matriz $W$ está na variável `M_bt` e a matriz $H$, na variável `M_tf`; modifique-o com os nomes que você escolheu para essas matrizes)

In [16]:
idx2words = np.array(tfidf.get_feature_names())

for idx in range(M_bt.shape[1]):
    top_words_indices = np.argsort(M_tf[idx])[::-1]
    print('Topic {}:'.format(idx))
    print('\t{}'.format('\n\t'.join(idx2words[top_words_indices[:10]])))

Topic 0:
	in
	2015
	2016
	2014
	audio
	reads
	library
	adult
	my
	book
Topic 1:
	children
	childrens
	kids
	picture
	childhood
	books
	middle
	juvenile
	grade
	animals
Topic 2:
	non
	nonfiction
	biography
	memoir
	memoirs
	history
	the
	of
	reading
	currently
Topic 3:
	fi
	sci
	fantasy
	science
	scifi
	sf
	owned
	default
	the
	epic
Topic 4:
	thriller
	mystery
	suspense
	crime
	mysteries
	default
	currently
	reading
	thrillers
	james
Topic 5:
	1001
	literature
	classics
	classic
	books
	american
	gilmore
	rory
	century
	school
Topic 6:
	paranormal
	fantasy
	romance
	series
	ya
	supernatural
	urban
	vampire
	magic
	young
Topic 7:
	historical
	club
	book
	the
	history
	currently
	reading
	favorites
	of
	war
Topic 8:
	graphic
	comics
	novels
	comic
	novel
	cmics
	and
	vol
	horror
	books
Topic 9:
	contemporary
	chick
	lit
	romance
	adult
	nora
	roberts
	own
	books
	favorites


**Etapa 3:** Use a matriz que descreve os livros em tópicos (matriz $H$) para fazer o `fit` do algoritmo [knn](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestNeighbors.html#sklearn.neighbors.NearestNeighbors)

** Tempo estimado para a atividade total: ** 5 minutos

In [17]:
from sklearn.neighbors import NearestNeighbors

In [ ]:
# Parâmetro para calcular o knn
n_neighbors = 40

In [18]:
knn = NearestNeighbors(n_neighbors=n_neighbors, metric='cosine')

In [19]:
knn.fit(M_bt)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='cosine',
         metric_params=None, n_jobs=1, n_neighbors=40, p=2, radius=1.0)

* você pode testar se está tudo ok, verificando que chamar o método `kneighbors` com a primeira das linhas (índice 0) da matriz retorna como mais próximo a própria linha

In [48]:
knn.kneighbors(M_bt[0])[1][0][0] == 0

True

**Etapa 4: ** Carregue os dados processados do outro notebook e encontre a matriz de usuário por tópicos

** Tempo estimado para a atividade total: ** 10 minutos

Primeiro, Será necessário carregar os seguintes arquivos:

* `ref_id2user_id`: `resources/data/processed/ref_id2user_id.joblib` 
* `ref_id2book_id`: `resources/data/processed/ref_id2book_id.joblib`
* `user_items_matrix`: `resources/data/processed/user_items_matrix.joblib`
* `user_ratings_df`: `resources/data/processed/user_ratings.csv`

Note que para os três primeiros, você usará a biblioteca joblib (que pode ser importada a partir do scikit-learn com `from sklearn.externals import joblib`), e para o último, a biblioteca `pandas`.

In [25]:
from sklearn.externals import joblib

In [32]:
ref_id2user_id = joblib.load('resources/data/processed/ref_id2user_id.joblib')
ref_id2book_id = joblib.load('resources/data/processed/ref_id2book_id.joblib')
unnormalized_user_items_matrix = joblib.load('resources/data/processed/unnormalized_user_items_matrix.joblib')
user_items_matrix = joblib.load('resources/data/processed/user_items_matrix.joblib')
test_user_items_matrix = joblib.load('resources/data/processed/test_user_items_matrix.joblib')
train_indices, test_indices = joblib.load('resources/data/processed/train_test_indices.joblib')
user_ratings_df = pd.read_csv('resources/data/processed/user_ratings.csv')

* após carregar os dados necessários, rode a seguinte célula

In [43]:
train_df, test_df = user_ratings_df.loc[train_indices], user_ratings_df.loc[test_indices]
user_id2ref_id = {user_id: ref_id for ref_id, user_id in ref_id2user_id.items()}
book_id2ref_id = {book_id: ref_id for ref_id, book_id in ref_id2book_id.items()}

* encontre agora a matriz de usuários por tópicos (dica: basta [multiplicar](https://docs.scipy.org/doc/numpy-1.14.1/reference/generated/numpy.dot.html) a matriz `user_items_matrix` pela matriz $H$)

In [26]:
M_ut = np.dot(user_items_matrix, csr_matrix(M_bt))

* essa matriz (a minha ficava na variável de nome `M_ut`) deverá ter dimensão `(5000, n_components)`

In [50]:
M_ut.shape == (5000, n_components)

True

**Etapa 5:** Dado um usuário (o índice da linha da matriz de usuário por tópicos) e um número $n > 0$, crie uma função que encontra as top $n$ recomendações de livros (lista de `ref_book_id`) para ele 

A ideia é que, ao ser passado o índice da linha (`ref_user_id`) que representa o usuário, usemos o índice `knn` gerado na etapa 3 para encontrar os $n$ livros (`ref_book_id`) mais próximos, que serão suas recomendações

** Tempo estimado para a atividade total: ** 5 minutos

In [37]:
def make_appropriate_conversions_from_ref_id(func):
    def modified_func(user_id, *args, **kwargs):
        predictions = func(user_id2ref_id[user_id], *args, **kwargs)
        return [ref_id2book_id[ref_book_id] for ref_book_id in predictions]
    return modified_func

In [39]:
@make_appropriate_conversions_from_ref_id
def get_recommendations_from_NMF(ref_user_id, n=10):
    distances, nearest_ref_book_ids = knn.kneighbors(M_ut[ref_user_id], n_neighbors=n)
    return nearest_ref_book_ids[0]

In [44]:
get_recommendations_from_NMF(5447, n=10)

[5957, 3513, 228, 3630, 5294, 1577, 1911, 2967, 3396, 957]

Acabamos de mimetizar a evolução do [sistema de recomendação do NYT](https://open.blogs.nytimes.com/2015/08/11/building-the-next-new-york-times-recommendation-engine/). Porém, o modelo de tópicos usado foi o [LDA](http://www.jmlr.org/papers/volume3/blei03a/blei03a.pdf). Veja um exemplo usando o LDA, que também usa uma ferramenta de visualização chamada `PyLDAvis` [nesse artigo do medium](https://medium.com/@sherryqixuan/topic-modeling-and-pyldavis-visualization-86a543e21f58).

## Indo além desses notebooks...

Para se aprofundar mais é preciso pensar em:

* como usar os dados que estão disponíveis para você como input desses sistemas de recomendação:
    * que tipo de dados existem? Por exemplo, existem avaliações de usuários para os itens que deseja recomendar?
    * existem muitos usuários novos todos os dias?

* como avaliar as recomendações:
    * importância de um baseline

    * parte qualitativa
    
    <img src="resources/imgs/exemplo_elo7.png" width=800>

    * parte quantitativa
        * [métricas básicas](https://heartbeat.fritz.ai/recommendation-systems-models-and-evaluation-84944a84fb8e)
        * [curso no Coursera sobre avaliação de sistemas de recomendação](https://pt.coursera.org/learn/recommender-metrics)
        * [usando CTR para medir a qualidade do retrieval](http://www.cs.cornell.edu/~tj/publications/radlinski_etal_08b.pdf)
    

* como ranquear as recomendações:
    * devemos promover certos itens?

* como colocar os modelos em produção